## Meshes, Functions, File I/O

This notebook will show some of the basic features of [Firedrake](https://www.firedrakeproject.org), which spyro is built upon. These are the kind of things you'd need to know to before you begin to model anything. First, we'll show how to use some of the builtin meshing capabilities in Firedrake, and then how to use meshes from an external mesher [SeismicMesh](https://github.com/krober10nd/SeismicMesh) and then talk a little bit about how Firedrake represents things on meshes using their concept of Functions. 

